In [1]:
import pandas as pd
import numpy as np

In [2]:
# read triple dataset
triple=pd.read_csv('/Users/ljystu/Downloads/qidpidtriples.train.full.2.tsv',sep="\t",names=['qid','positive pid','negative pid'])

In [3]:
# read queries
topics=pd.read_csv('/Users/ljystu/Library/CloudStorage/OneDrive-DelftUniversityofTechnology/IR/IRCore/queries/queries.train.tsv',sep='\t',names=['qid','query'])


In [4]:
# read train features
train=pd.read_csv('train_new.csv')
train=pd.merge(train,topics,on='query')

# read relevance=1
positive_relevance=triple[['qid','positive pid']].drop_duplicates()
positive_relevance= positive_relevance.reindex(columns=['qid', 'positive pid', 'target'],fill_value=1)
positive_relevance.columns=['qid','pid','target']

In [5]:
# read relevance=0
negative_relevance=triple[['qid','negative pid']]

negative_relevance=negative_relevance.reindex(columns=['qid','negative pid','target'],fill_value=0)
negative_relevance.columns=['qid','pid','target']

In [6]:
# take negative and positive labels of 1:10
sampled_negative_relevance=negative_relevance.sample(n=len(positive_relevance)*10)

In [7]:
qrels=pd.concat([positive_relevance,sampled_negative_relevance])
qrels

,qid,pid,target
0,1000094,5399011,1
999,1000684,6133670,1
1607,1000938,5203582,1
2606,1001876,2354621,1
3605,1002233,5049159,1
...,...,...,...
74531363,247682,5246302,0
175898478,401274,8648677,0
250885409,1033377,6346955,0
361056607,515585,1891094,0


In [8]:
train

,Unnamed: 0,index,qid_x,docid,docno,rank,score,query,features,qid_y
0,0,0,1,674182,674182,0,25.165898,define extreme,[25.16589805 13.73858583 12.78250003 3. ],121352
1,1,1,1,1702145,1702145,1,23.961474,define extreme,[23.96147395 13.08125554 12.06740281 2. ],121352
2,2,2,1,792622,792622,2,23.796681,define extreme,[23.79668071 12.9910812 11.95633148 3. ],121352
3,3,3,1,7282917,7282917,3,23.168934,define extreme,[23.16893426 12.6483728 11.59384193 3. ],121352
4,4,4,1,8807233,8807233,4,22.973981,define extreme,[22.97398086 12.54215475 11.49753609 4. ],121352
...,...,...,...,...,...,...,...,...,...,...
960811,1101566,95,1,4169488,4169488,95,21.708582,what is the minimum number of days for an info...,[21.7085823 11.96094963 12.14951632 20. ],832872
960812,1101567,96,1,7204837,7204837,96,21.708383,what is the minimum number of days for an info...,[21.70838256 18.52964019 15.67249025 10. ],832872
960813,1101568,97,1,1154364,1154364,97,21.696975,what is the minimum number of days for an info...,[21.69697474 15.43666219 13.18653307 8. ],832872
960814,1101569,98,1,4440325,4440325,98,21.677164,what is the minimum number of days for an info...,[21.67716418 18.4991468 15.99255852 14. ],832872


In [9]:
train=train[['qid_y','docid','features']]
train.columns=['qid','pid','features']
merged=pd.merge(qrels,train,how='inner',on=['pid','qid'])

In [10]:
merged

,qid,pid,target,features
0,1043711,2988261,1,[31.79691149 19.56379719 18.11058261 8. ]
1,231709,2116375,1,[62.70007435 36.06257036 32.41125656 19. ]
2,334556,2251059,1,[33.330516 19.41301388 16.9644778 4. ]
3,451601,5889185,1,[36.42437422 19.93415333 16.96244524 7. ]
4,542138,1697949,1,[17.96219802 9.80077207 8.51344013 1. ]
...,...,...,...,...
8123,592018,281931,0,[41.06770105 23.78156317 21.0035478 13. ]
8124,970763,5687618,0,[30.96604176 17.00910283 14.79368029 4. ]
8125,854207,8378167,0,[40.74223511 22.25314466 20.26567377 9. ]
8126,122479,6385802,0,[25.09575325 13.6942134 12.28965602 2. ]


In [11]:
merged=merged.sample(frac=1)
merged['target'].value_counts()

0    4542
1    3586
Name: target, dtype: int64

In [46]:
merged=merged.astype(str)
with open('triples_train.txt','a') as f:    
    #write features
    for row in merged.iterrows():

        temp=row[-1].iloc[-1]
        features=temp[1:-2].split()

        line=""+row[1].iloc[2]+" qid:"+row[1].iloc[0]+" 1:"+features[0]+" 2:"+features[1]+" 3:"+features[2]+" #docid="+row[1].iloc[1]
        f.writelines(line+"\n")
f.close()